<style TYPE="text/css">
code.has-jax {font: inherit; font-size: 100%; background: inherit; border: inherit;}
</style>
<script type="text/x-mathjax-config">
MathJax.Hub.Config({
    tex2jax: {
        inlineMath: [['$','$'], ['\\(','\\)']],
        skipTags: ['script', 'noscript', 'style', 'textarea', 'pre'] // removed 'code' entry
    }
});
MathJax.Hub.Queue(function() {
    var all = MathJax.Hub.getAllJax(), i;
    for(i = 0; i < all.length; i += 1) {
        all[i].SourceElement().parentNode.className += ' has-jax';
    }
});
</script>
<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=TeX-AMS_HTML-full"></script>
<h1>Lab of Statistical Signal Processing - Deep Neural Network for CS-based signal reconstruction on STM32 MCU board</h1>

<h2>Project Requirement</h2>
<ul>
    <li>System Workbench for STM32 (or equivalent IDE)</li>
    <li>STCube MX v5.2.0</li>
    <li>STCube MX AI package v4.1.0</li>
    <li>Python v3.7</li>
    <li>Tensorflow v2.0</li>
    <li>Jupyter Notebook</li>
    <li>SMT32 Board - NucleoH743ZI2 and STM32F4DISCO have been used for this demo</li>
</ul>

<h2>Compressed Sensing Basics</h2>
<p>CS hinges on the assumption that
x is $\kappa$-sparse, i.e., in the simplest possible setting, that an
orthonormal matrix $S$ exists (whose columns are the vectors
of the sparsity basis) such that when we express $x=S\xi$, then
the vector $\mathrm{\xi = (\xi_{1},...,\xi_{n-1})}$ does not contain more than $\kappa < n$ non-zero entries.
The fact that $x$ depends only on a number of scalars that
is less than its sheer dimensionality hints at the possibility
of compressing it. CS does this by applying a linear operator
$\mathcal{L_{\mathrm{A}}} : \mathbb{R^m} \mapsto \mathbb{R^n}$  depending on the acquisition (or encoding)
matrix $A \in \mathbb{R^{m\times n}}$ with $m < n$ and defined in such a way that
$x \in \mathbb{R^n}$ can be retrieved from $y = \mathcal{L_{\mathrm{A}}(x)} \in \mathbb{R^m}$. The ratio
$n/m$ is the compression ratio and will be indicated by CR.
<cite>[Mangia, Mauro & Prono, Luciano & Pareschi, Fabio & Rovatti, Riccardo & Setti, Gianluca. (2019). Deep Neural Oracles for Short-window Optimized Compressed Sensing of Biosignals. 10.36227/techrxiv.10049843.v2. ]</cite></p>

<p>More specifically, in this demo a serial communication has been established between the MCU and Python enironment. A master computer feeds y_test data to the MCU which in turn computes the output $\hat{s}$ from the neural network and sends back such data to the computer. Finally, RSNR and the reconstructed input $\hat{x}$ are computed by the Jupyter Notebook.</p>

![Figure 1](Cattura.png "Figure 1")
    
    
<h2>MCU Preparation</h2>
<p>After having installed STCube MX version 5.2 and AI extension version 4.1 on a virtual machine with Windows 7 (Neither "Validation on Desktop" nor "Validation on Target" tasks appear to work on Windows 10), I have followed these steps:
<ol>
    <li>Click on "Access Board Selector", choose the appropriate device and click on "Start Project".</li>
    <li>Enable UART peripheral: USART3 peripheral of NUCLEO-144 board is directly connected to the ST-LINK therefore just a USB cable wil be needed for both programming and communication. Select USART2 from "Connectivity" dropdown menu and select "Asynchronous".</li>
    <li>Cick on "Additional Software" -> "STMicroelectronics.X_CUBE_AI" -> "Validation" (or "Application Template" to generate the code)</li>
    <li>Open Artificial Intelligence menu from the left-hand side, select the UART peripheral under "Platform Setting" previously enabled and click on "Add Network". Choose "Keras", "Saved Model" and upload the h5 format file containing the trained Neural Network.</li>
    <li>The model can be accurately examined with the options "Analyze", "Validate on Desktop", "Validate on target". Pay attention to select the correct COM port associated to the ST-LINK. This can be checked from "Control Panel" in Windows.</li>
    <li>Choose the adopted IDE (for this demo SystemWorkbench for STM32) in "Project Manager" options and run "Generate Code" (Remember to switch mode from Validation to Application Template, see point 3)</li>
</ol>
The AI project with the Neural Network can now be moved to the actual OS and the Virtual Machine be turned off. Some modification of the code are needed in order for the serial communication to work properly.
<ul>
    <li>Initialize the UART peripheral by calling this procedure inside main.c</li>
        
```c
MX_USART3_UART_Init();
```
   <li>Define globally the following variables inside app_x-cube-ai.c</li>
   
```c
UART_HandleTypeDef huart3;
ai_float outdata[AI_NETWORK_OUT_1_SIZE];
uint8_t shat[AI_NETWORK_OUT_1_SIZE];
ai_float threshold = 0.0352120689269106; //O_min threshold
```

   <li>Modify MX_X_CUBE_AI_Process() function</li>

```c
void MX_X_CUBE_AI_Process(void)
{
    /* USER CODE BEGIN 1 */
	int nb_run = 100;
    AI_ALIGNED(4)
	//INPUT AND OUTPUT BUFFERS
    static ai_i8 out_data[AI_NETWORK_OUT_1_SIZE_BYTES];
    static ai_u8 in_data[AI_NETWORK_IN_1_SIZE_BYTES];

    /* Perform nb_rub inferences (batch = 1) */
    while (--nb_run) {
        /* ---------------------------------------- */
        /* Data generation and Pre-Process          */
        /* ---------------------------------------- */
    	//FILL THE INPUT BUFFER
    	while(HAL_UART_Receive(&huart3, in_data, sizeof(in_data), HAL_MAX_DELAY)!=HAL_OK);
        /* Perform the inference */
        aiRun(in_data, out_data);
        for (ai_size i=0;  i < AI_NETWORK_OUT_1_SIZE; i++ ){
        	//CASTING TO AI_FLOAT THE OUTPUT OF THE NN
        	outdata[i] = ((ai_float *)out_data)[i];
        	//COMPUTING S_HAT
        	if(outdata[i] > threshold) shat[i] = 0x01;
        	else shat[i] = 0x00;
        }
        //TRANSMIT S_HAT
        while(HAL_UART_Transmit(&huart3,shat,sizeof(shat),0xFFFF)!=HAL_OK);
    }
    /* USER CODE END 1 */
}
```   
</ul>
The MCU is now ready to communicate with the computer. If the communication does not work make sure if the baud rate specified in the COM port setting in the control panel is consistent with the value present in UART_Init() procedure.
</p>

<h2>Script Python</h2>
<p>
The following script essentially imports the dataset, establishes a serial connection with the MCU and computes support mismatch and Recontruction SNR with the $\hat{s}$ vectors provided by the microcontroller.
</p>

In [6]:
from scipy import signal
from scipy import io
from scipy import random
import os
import serial
import sys
import numpy as np
import time
import struct
from dnnCS_functions import *

In [7]:
# The data, split between train and test sets:
# Load y_test float32 type
data = io.loadmat('float_y.mat')
y_test = data['ans'][:32,:].T
# sparsity basis
D = io.loadmat('D.mat')['D']
# sensing matrix
A = io.loadmat('A.mat')['A'][:32,:]
# dataset
data = io.loadmat('test_set.mat')
X_test, S_test = data['X_test'], data['S_test']
B = A@D

In [8]:
#Check the correct COMx port associated to the STLink and choose a consistend baud rate
port = 'COM6'
baud = 115200  
ser = serial.Serial(port, baud, timeout=0)

In [9]:
for i in range(100):
    for j in range(32):
        #Iterations over the 32 elements of an y_test vector
        ser.write(bytearray(y_test[i][j]))
        time.sleep(.1)
    ser.flushOutput()
    #Reading the sent data. Note the output is an hexadecimal string of 128 bytes filled with 0s and 1s
    reading = ser.readline()
    ser.flushOutput()
    #Converting the data into an numpy boolean array type, required by xi_estimation function
    s_hat = struct.unpack('????????????????????????????????????????????????????????????????', reading)
    s_hat = np.asarray(s_hat)
    xi = xi_estimation(y_test[i], s_hat, B)
    x_hat = D@xi
    print('RSNR = {r:6.2f} with support missmatch = {ms:d}'.format(r=RSNR(X_test[i],x_hat),ms=int(sum(np.abs(S_test[i]-s_hat)))))

RSNR = 144.98 with support missmatch = 1
RSNR = 146.56 with support missmatch = 0
RSNR = 143.85 with support missmatch = 2
RSNR = 147.20 with support missmatch = 1
RSNR = 139.87 with support missmatch = 1
RSNR = 144.75 with support missmatch = 2
RSNR = 142.29 with support missmatch = 1
RSNR = 145.90 with support missmatch = 1
RSNR =  40.58 with support missmatch = 2
RSNR = 143.28 with support missmatch = 0
RSNR = 146.28 with support missmatch = 0
RSNR = 140.77 with support missmatch = 2
RSNR =  44.61 with support missmatch = 4
RSNR = 139.73 with support missmatch = 1
RSNR = 148.70 with support missmatch = 0
RSNR = 146.02 with support missmatch = 3
RSNR = 140.36 with support missmatch = 2
RSNR = 146.48 with support missmatch = 0
RSNR = 140.94 with support missmatch = 1
RSNR = 151.36 with support missmatch = 0
RSNR = 145.56 with support missmatch = 3
RSNR = 143.78 with support missmatch = 2
RSNR = 143.05 with support missmatch = 0
RSNR = 140.50 with support missmatch = 3
RSNR = 144.69 wi

In [10]:
ser.close()